In [254]:
import yaml
import pandas as pd
import os
import numpy as np

In [255]:
## Variable
config_file_path = "config.yml"

In [256]:
# functions
def config_loading(config_path:str):
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

def build_output_paths(config):
    """
    Build full paths for output files based on the configuration.

    Args:
        config (dict): Configuration dictionary with keys:
            - root_path: Root directory path.
            - output_folder: Folder where output files will be saved.
            - filtered_data_filename: Filename for filtered data.
            - no_null_imputed_data_filename: Filename for no-null imputed data.
            - scaled_data_filename: Filename for scaled data.

    Returns:
        dict: A dictionary with keys 'filtered_data_path', 'no_null_imputed_data_path',
              and 'scaled_data_path', containing the full paths for the respective files.
    """
    # Get the root path and output folder
    root_path = config["root_path"]
    output_folder = config["output_folder"]

    # Ensure the output folder exists
    output_path = os.path.join(root_path, output_folder)
    os.makedirs(output_path, exist_ok=True)

    # Build full paths for the output files
    paths = {
        "filtered_data_path": os.path.join(
            output_path, config["filtered_data"]
        ),
        "no_null_imputed_data_path": os.path.join(
            output_path, config["no_null_imputed_data"]
        ),
        "scaled_data_path": os.path.join(output_path, config["scaled_data"]),
    }

    return paths

def date_dv_columns_check(data: pd.DataFrame, config: dict):
    required_columns = [config['date_column'], config['dv_column']] + config['data_prep_group_var']
    missing_columns = [col for col in required_columns if col not in data.columns]

    if missing_columns:
        raise ValueError(f"The following required columns are missing from the DataFrame: {missing_columns}")
    else:
        print("All required columns are present in the DataFrame.")

def data_date_conversion(data:pd.DataFrame, config:dict):
    data[config['date_column']] = pd.to_datetime(data[config['date_column']], format=config['date_format'])
    return data

def idv_list_loading(config:dict):
    idv = pd.read_csv(config['idv_list'])
    return idv

def check_idv_columns_in_data(data:pd.DataFrame, idv_list:pd.DataFrame, column_name:str, config):
    """
    Checks if all columns in df1 are present as rows in a specified column of df2.

    Parameters:
    - data (pd.DataFrame): The first DataFrame whose columns need to be checked.
    - idv_list (pd.DataFrame): The second DataFrame with the reference column.
    - column_name (str): The column in df2 that should contain all column names of df1.

    Returns:
    - None: If all columns are found, the function silently passes.

    Raises:
    - ValueError: If any columns are missing, it raises an error with the missing columns.
    """
    # Get the list of columns from df1
    df1_columns = set(data.columns)

    # Get the unique values in the specified column of df2
    df2_values = set(idv_list[column_name])

    # Find the missing columns
    missing_columns = df2_values - df1_columns

    # Raise an error if there are missing columns
    if missing_columns:
        raise ValueError(f"The following independent variables are missing in data: {missing_columns}")
    else:
        print("All independent variables in idv_list are present in the data.")
    required_columns = [config['date_column'], config['dv_column']] + config['data_prep_group_var']
    return data[required_columns + list(df2_values)]

def data_loading(config:dict):

    input_data = pd.read_csv(config['input_data'])
    date_dv_columns_check(input_data, config)
    data = data_date_conversion(input_data, config)
    idv_list = idv_list_loading(config)
    data = check_idv_columns_in_data(data, idv_list, "idv", config)

    return data, idv_list

def column_arrangement(config: dict, idv_list:pd.DataFrame):
    sorted_idv_list = sorted(idv_list['idv'].tolist())
    sorted_idv_list
    column_arrangement = [config['date_column']] + config['data_prep_group_var'] + sorted_idv_list + [config['dv_column']]
    return column_arrangement

def filter_by_date_range(data:pd.DataFrame, config:dict):
    """Filtering date range for the data processing and further analysis

    Args:
        data (DataFrame): Harmonized_processed_data to filter out the date range
        config (dict): configuration dictionary
    Returns:
        DataFrame: Data with filtered date range
    """
    data["date"] = pd.to_datetime(data["date"], utc=False)

    # Print the minimum and maximum date values for verification
    print("Minimum date:", data["date"].min(skipna=True))
    print("Maximum date:", data["date"].max(skipna=True))

    # Define the date range from run_config
    date1 = pd.to_datetime(config['start_date'], format="%Y-%m-%d")
    date2 = pd.to_datetime(config['end_date'], format="%Y-%m-%d")

    # Filter the DataFrame based on the date range
    data = data[(data["date"] >= date1) & (data["date"] <= date2)]
    if data[config['dv_column']].isna().sum() != 0:
        raise ValueError(f"The dependent variable {config['dv_column']} is having null values")
    return data

### preprocessing functions ####

def cap_values(df, col, min_val, max_val):
    df[col] = df[col].astype(float).clip(lower=min_val, upper=max_val)
    return df


def get_unique_combinations(df: pd.DataFrame, columns: list):
    """
    Get unique combinations of values in the specified columns of a DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame to extract combinations from.
    - columns (list): A list of column names to consider.

    Returns:
    - list of dict: Each dictionary represents a unique combination of column-value pairs.
    """
    # Check if the specified columns exist in the DataFrame
    missing_columns = [col for col in columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"The following columns are missing from the DataFrame: {missing_columns}")

    # Extract unique rows for the specified columns
    unique_combinations = df[columns].drop_duplicates().to_dict(orient="records")

    return unique_combinations


def drop_rows_with_nulls(df: pd.DataFrame, group_by_columns: list, null_threshold: float):
    """
    Drop rows for specific combinations of columns where more than 50% of the 'value' column is null.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - group_by_columns (list): List of columns to group by.

    Returns:
    - pd.DataFrame: The DataFrame with rows removed based on the condition.
    """
    # Step 1: Group by specified columns
    grouped = df.groupby(group_by_columns)

    # Step 2: Calculate the percentage of nulls for each group
    null_percentage = grouped['value'].apply(lambda x: x.isnull().mean())

    # Step 3: Identify groups with more than 50% null values
    groups_to_drop = null_percentage[null_percentage > null_threshold].index

    # Step 4: Filter out rows belonging to the identified groups
    filtered_df = df[~df.set_index(group_by_columns).index.isin(groups_to_drop)]

    return filtered_df


def impute_groups(df: pd.DataFrame, group_by_columns: list, null_threshold: float, imputation_method: str = 'mean'):
    """
    Drop rows for specific combinations of columns where more than the specified percentage of the 'value' column is null,
    and impute missing values for groups with less than the threshold.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - group_by_columns (list): List of columns to group by.
    - null_threshold (float): The threshold of null percentage for which rows will be dropped.
    - imputation_method (str): The method for imputing missing values ('mean', 'median', 'mode').

    Returns:
    - pd.DataFrame: The DataFrame with rows removed and missing values imputed.
    """
    # Step 1: Group by specified columns
    grouped = df.groupby(group_by_columns)

    # Step 2: Calculate the percentage of nulls for each group
    null_percentage = grouped['value'].apply(lambda x: x.isnull().mean())

    # Step 3: Identify groups with more than the null threshold percentage of null values
    groups_to_drop = null_percentage[null_percentage > null_threshold].index
    groups_to_drop_list = groups_to_drop.to_list()

    print(groups_to_drop_list)
    # Step 4: Filter out rows belonging to the identified groups
    df_filtered = df[~df.set_index(group_by_columns).index.isin(groups_to_drop)]

    # Step 5: Impute missing values for the remaining groups (those with less than the threshold null percentage)
    for group, group_df in df_filtered.groupby(group_by_columns):
        if group not in groups_to_drop:
            if imputation_method == 'mean':
                fill_value = group_df['value'].mean()
            elif imputation_method == 'median':
                fill_value = group_df['value'].median()
            elif imputation_method == 'mode':
                fill_value = group_df['value'].mode()[0]
            else:
                raise ValueError(f"Unsupported imputation method: {imputation_method}")

            # Impute the missing values in the group
            df_filtered.loc[group_df.index, 'value'] = group_df['value'].fillna(fill_value)

    return df_filtered

# Scaling Function
def scale_metrics(melted_data, idv_list, config):
    def apply_scaling(group):
        metric_name = group['metric'].iloc[0]
        # Skip scaling for the dependent variable
        if metric_name == config['dv_column']:
            group['value'] = group['value']
            return group
        if config['scaling'] == 'minmax':
            # Min-Max Scaling: Scale between 0 and 1
            min_val = group['value'].min()
            max_val = group['value'].max()
            group['value'] = (group['value'] - min_val) / (max_val - min_val)

        elif config['scaling'] == 'standard':
            # Standard Scaling: Scale with mean and standard deviation
            mean_val = group['value'].mean()
            std_dev = group['value'].std()
            group['value'] = (group['value'] - mean_val) / std_dev

        elif config['scaling'] == 'custom':
            # Custom Scaling: Use min and max from df2
            custom_min = idv_list.loc[idv_list['idv'] == metric_name, 'min'].values[0]
            custom_max = idv_list.loc[idv_list['idv'] == metric_name, 'max'].values[0]
            print((metric_name, custom_max, custom_min))
            group['value'] = (group['value'] - custom_min) / (custom_max - custom_min)

        else:
            raise ValueError("Invalid scaling method. Choose 'minmax', 'standard', or 'custom'.")

        return group

    # Group by 'metric' and apply scaling
    scaled_df = melted_data.groupby('metric', group_keys=False).apply(apply_scaling)
    return scaled_df



In [257]:
config = config_loading(config_file_path)
data, idv_list = data_loading(config)
filtered_data = filter_by_date_range(data, config)

All required columns are present in the DataFrame.
All independent variables in idv_list are present in the data.
Minimum date: 2017-01-07 00:00:00
Maximum date: 2025-01-11 00:00:00


In [258]:
config

{'input_data': 'D:\\BRAND_HUB_PROJECT\\brandhub-capability\\src\\BHC_Capability\\data\\input_data_refined_4.csv',
 'idv_list': 'D:\\BRAND_HUB_PROJECT\\brandhub-capability\\src\\BHC_Capability\\data\\idv_list_.csv',
 'granularity': 'weekly',
 'data_prep_group_var': ['brand', 'category'],
 'date_column': 'date',
 'date_format': '%d-%m-%Y',
 'start_date': '2022-08-01',
 'end_date': '2024-06-01',
 'dv_column': 'market_share',
 'null_percentage': 0.5,
 'scaling': 'custom',
 'cfa_sampling_seeding': [2, 3, 5, 7, 11, 13, 17, 19],
 'root_path': './',
 'output_folder': 'output',
 'filtered_data': 'filtered_data.csv',
 'no_null_imputed_data': 'no_null_imputed_data.csv',
 'scaled_data': 'scaled_data.csv'}

In [259]:
#Data Preprocessing
melted_df = pd.melt(
    filtered_data,
    id_vars=['date', 'brand', 'category'],  # Columns to keep as-is
    var_name='metric',  # Name of the new column for melted variable names
    value_name='value'  # Name of the new column for melted values
)
no_null_imputed_data = impute_groups(melted_df, ['brand', 'category', 'metric'], .5)
scaled_data = scale_metrics(no_null_imputed_data, idv_list, config)


[("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_always_seems_to_be_doing_something_new_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_creates_excitement_and_eagerness_at_mealtime_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_creates_playful_moments_with_my_cat_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_has_a_taste_my_cat_enjoys_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_has_a_taste_my_dog_enjoys_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_has_craveable_treats_that_my_cat_comes_running_for_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_has_the_most_satisfying_and_hearty_dog_treats_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_has_treats_that_contain_wholesome_ingredients_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_has_treats_that_have_an_appealing_package_mean'), ("BEGGIN'", 'DOG FOOD', 'directions_brand_attributes_is_a_brand_i_do_not_trust_to_feed_to_my_

C:\Users\cilvo.simon\AppData\Local\Temp\ipykernel_28092\88934075.py:267: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  scaled_df = melted_data.groupby('metric', group_keys=False).apply(apply_scaling)


### CFA analysis

In [260]:
scaled_data

,date,brand,category,metric,value
0,2022-08-13,BEGGIN',DOG FOOD,market_share,0.008066
1,2022-08-13,PEDIGREE,DOG FOOD,market_share,0.070409
2,2022-08-13,TEMPTATIONS,CAT FOOD,market_share,0.055228
3,2022-08-20,BEGGIN',DOG FOOD,market_share,0.007958
4,2022-08-20,BLUE BUFFALO,DOG FOOD,market_share,0.074323
...,...,...,...,...,...
38395,2024-02-24,BLUE BUFFALO,CAT FOOD,directions_strategic_measures_brand_interest_i...,0.659667
38396,2024-02-24,TEMPTATIONS,CAT FOOD,directions_strategic_measures_brand_interest_i...,0.979523
38397,2024-03-02,PEDIGREE,DOG FOOD,directions_strategic_measures_brand_interest_i...,0.264890
38398,2024-03-09,BEGGIN',DOG FOOD,directions_strategic_measures_brand_interest_i...,0.250236


In [262]:
group_list = [config['date_column']] + config['data_prep_group_var']
for group, group_df in scaled_data.groupby(['brand', 'category']):
    if group == ("BEGGIN'", 'DOG FOOD'):
        cfa_data = group_df.pivot(index=group_list, columns='metric', values='value').reset_index()
        group_value_tuple = group

In [263]:
pillar_idv_dict = idv_list.groupby('equity_pillar')['idv'].apply(list).to_dict()

In [264]:
pillar = 'awareness'
col_available = [col for col in cfa_data if col in pillar_idv_dict[pillar]]
cfa_data_filtered = cfa_data[col_available]
factor_str = f"{pillar}_pillar =~ " + " + ".join(col_available)



In [265]:
# Generate samples based on seeds
cfa_data_filtered_list = []
for seed in config['cfa_sampling_seeding']:
    sampled_df = cfa_data_filtered.sample(frac=0.95, random_state=seed)
    cfa_data_filtered_list.append(sampled_df)

In [266]:
import semopy as sp
from semopy import Model


def cfa_py(fa_str, scaled_data):

    model = Model(fa_str, cov_diag=False)
    model.fit(scaled_data, solver="L-BFGS-B")  # , estimator="GLS")

    # Retrieve fit statistics (method may vary)
    stats = sp.calc_stats(model)

    try:
        fit_indices = model.fit_stats()
    except AttributeError:
        # If fit_stats() is not available, check other attributes
        fit_indices = (
            model.statistics_ if hasattr(model, "statistics_") else {}
        )
    cfa_fit_indices = pd.DataFrame(
        fit_indices.items(), columns=["fitmeasure", "value"]
    )
    cfa_fit_indices = cfa_fit_indices[
        cfa_fit_indices["fitmeasure"].isin(["cfi", "tli", "rmsea"])
    ]
    cfa_fit_indices["value"] = cfa_fit_indices["value"].astype(float)

    cfa_fit_indices_t = cfa_fit_indices.set_index("fitmeasure").T

    # Extract parameter estimates
    cfa_estimates = model.inspect()

    # CFA summary table
    cfa_summary = pd.concat([cfa_estimates, cfa_fit_indices_t], axis=1)

    # Store the results
    cfa_summary["factor_str"] = fa_str
    # cfa_summary["Seed"] = seed
    cfa_summary["cfi"] = stats.loc["Value"]["CFI"]
    cfa_summary["tli"] = stats.loc["Value"]["TLI"]
    cfa_summary["rmsea"] = stats.loc["Value"]["RMSEA"]
    cfa_summary = cfa_summary.rename(
        columns={
            "lval": "rhs",
            "rval": "lhs",
            "Estimate": "est.std",
            "Std. Err": "se",
            "z-value": "z",
            "p-value": "pvalue",
        }
    )
    cfa_summary.replace({"op": {"~": "=~"}}, inplace=True)
    columns_list = ["rhs", "op", "lhs", "est.std", "se", "z", "pvalue", "factor_str", "cfi", "tli", "rmsea"]

    cfa_summary = cfa_summary[columns_list]

    return cfa_summary

In [267]:
fit_cfa_df = cfa_py(factor_str, cfa_data_filtered).dropna(subset = ['op'])
for i, x in enumerate(config['data_prep_group_var']):
    fit_cfa_df[x] = group_value_tuple[i]

fit_cfa_df

c:\Users\cilvo.simon\AppData\Local\miniforge3\Lib\site-packages\semopy\stats.py:383: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt((chi2 / dof - 1) / (model.n_samples - 1))


,rhs,op,lhs,est.std,se,z,pvalue,factor_str,cfi,tli,rmsea,brand,category
0,directions_awareness_total_awareness_net_mentions,=~,awareness_pillar,1.000000,-,-,-,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD
1,directions_awareness_unaided_awareness_net_men...,=~,awareness_pillar,1.002990,0.0208,48.219697,0.0,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD
2,social_share_of_total_buzz_post,=~,awareness_pillar,0.085560,0.004339,19.716983,0.0,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD
3,awareness_pillar,~~,awareness_pillar,0.018327,0.002651,6.914451,0.0,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD
4,directions_awareness_total_awareness_net_mentions,~~,directions_awareness_total_awareness_net_mentions,0.000000,0.000167,0.0,1.0,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD
5,directions_awareness_unaided_awareness_net_men...,~~,directions_awareness_unaided_awareness_net_men...,0.000614,0.00019,3.235352,0.001215,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD
6,social_share_of_total_buzz_post,~~,social_share_of_total_buzz_post,0.000032,0.000005,6.698613,0.0,awareness_pillar =~ directions_awareness_total...,0.81151,NaN,inf,BEGGIN',DOG FOOD


In [268]:
def process_cfa_samples(cfa_py, factor_str, cfa_data_filtered, config, group_tuple):
    """
    Processes CFA samples, fits CFA to each sample, and concatenates the results.

    Args:
        cfa_py (function): Function to perform CFA.
        factor_str (str): The CFA factor string.
        cfa_data_filtered (DataFrame): The original filtered DataFrame.
        config (dict): Configuration dictionary with sampling seeds and group variables.
        group_tuple (tuple): Tuple containing values for `data_prep_group_var`.

    Returns:
        DataFrame: Concatenated results with additional columns.
    """
    cfa_data_filtered_list = []

    # Perform sampling and fit CFA for each sample
    for seed in config['cfa_sampling_seeding']:
        # Sample 95% of the data
        sampled_df = cfa_data_filtered.sample(frac=0.95, random_state=seed)

        # Fit CFA and drop rows with missing 'op' values
        fit_cfa_df = cfa_py(factor_str, sampled_df).dropna(subset=['op'])

        # Add group variables as columns
        for i, x in enumerate(config['data_prep_group_var']):
            fit_cfa_df[x] = group_tuple[i]

        fit_cfa_df['seed'] = seed

        # Append the processed DataFrame to the list
        cfa_data_filtered_list.append(fit_cfa_df)

    # Concatenate all results
    concatenated_results = pd.concat(cfa_data_filtered_list, ignore_index=True)

    return concatenated_results


In [271]:
def perform_cfa_analysis(scaled_data, idv_list, config, cfa_py):
    """
    Performs CFA analysis across multiple groups, equity pillars, and samples.
    """
    group_list = [config['date_column']] + config['data_prep_group_var']
    final_results = []  # To store results from all groups and equity pillars
    pillar_idv_dict = idv_list.groupby('equity_pillar')['idv'].apply(list).to_dict()
    for group, group_df in scaled_data.groupby(['brand', 'category']):

        # Pivot the data
        cfa_data = group_df.pivot(index=group_list, columns='metric', values='value').reset_index()
        group_value_tuple = group  # Capture group values for column assignment

        for pillar in pillar_idv_dict.keys():
            # Filter columns available for the current pillar
            col_available = [col for col in cfa_data if col in pillar_idv_dict[pillar]]

            if col_available:  # Proceed only if there are relevant columns
                # Filter data for CFA
                cfa_data_filtered = cfa_data[col_available]

                # Create factor string
                factor_str = f"{pillar} =~ " + " + ".join(col_available)

                # Process CFA samples and collect results
                results = process_cfa_samples(
                    cfa_py=cfa_py,
                    factor_str=factor_str,
                    cfa_data_filtered=cfa_data_filtered,
                    config=config,
                    group_tuple=group_value_tuple
                )
                final_results.append(results)

    # Concatenate all results from all groups and pillars
    return pd.concat(final_results, ignore_index=True)

In [272]:
perform_cfa_analysis(scaled_data, idv_list, config, cfa_py)

c:\Users\cilvo.simon\AppData\Local\miniforge3\Lib\site-packages\semopy\stats.py:383: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt((chi2 / dof - 1) / (model.n_samples - 1))
c:\Users\cilvo.simon\AppData\Local\miniforge3\Lib\site-packages\semopy\stats.py:383: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt((chi2 / dof - 1) / (model.n_samples - 1))
c:\Users\cilvo.simon\AppData\Local\miniforge3\Lib\site-packages\semopy\stats.py:383: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt((chi2 / dof - 1) / (model.n_samples - 1))
c:\Users\cilvo.simon\AppData\Local\miniforge3\Lib\site-packages\semopy\stats.py:383: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt((chi2 / dof - 1) / (model.n_samples - 1))
c:\Users\cilvo.simon\AppData\Local\miniforge3\Lib\site-packages\semopy\stats.py:383: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt((chi2 / dof - 1) / (mo

,rhs,op,lhs,est.std,se,z,pvalue,factor_str,cfi,tli,rmsea,brand,category,seed
0,directions_funnel_metrics_advocacy_t2b_buyers,=~,advocacy,1.000000,-,-,-,advocacy =~ directions_funnel_metrics_advocacy...,0.688131,NaN,inf,BEGGIN',DOG FOOD,2
1,directions_strategic_measures_brand_love_index,=~,advocacy,0.227565,0.068392,3.327383,0.000877,advocacy =~ directions_funnel_metrics_advocacy...,0.688131,NaN,inf,BEGGIN',DOG FOOD,2
2,social_percent_positive_neutral,=~,advocacy,-0.079943,0.028482,-2.806788,0.005004,advocacy =~ directions_funnel_metrics_advocacy...,0.688131,NaN,inf,BEGGIN',DOG FOOD,2
3,advocacy,~~,advocacy,0.049732,0.015477,3.213364,0.001312,advocacy =~ directions_funnel_metrics_advocacy...,0.688131,NaN,inf,BEGGIN',DOG FOOD,2
4,directions_funnel_metrics_advocacy_t2b_buyers,~~,directions_funnel_metrics_advocacy_t2b_buyers,0.000176,0.013594,0.012962,0.989658,advocacy =~ directions_funnel_metrics_advocacy...,0.688131,NaN,inf,BEGGIN',DOG FOOD,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,ratings_reviews_review_sentiment_score_average,~~,ratings_reviews_review_sentiment_score_average,0.000381,0.000057,6.740209,0.0,product_feedback =~ ratings_reviews_good_exper...,0.954372,0.936121,0.226292,TEMPTATIONS,CAT FOOD,19
5004,ratings_reviews_review_sentiment_score_health_...,~~,ratings_reviews_review_sentiment_score_health_...,0.033746,0.005021,6.721637,0.0,product_feedback =~ ratings_reviews_good_exper...,0.954372,0.936121,0.226292,TEMPTATIONS,CAT FOOD,19
5005,ratings_reviews_review_sentiment_score_ingredi...,~~,ratings_reviews_review_sentiment_score_ingredi...,0.062984,0.009338,6.745307,0.0,product_feedback =~ ratings_reviews_good_exper...,0.954372,0.936121,0.226292,TEMPTATIONS,CAT FOOD,19
5006,ratings_reviews_review_sentiment_score_pet_enj...,~~,ratings_reviews_review_sentiment_score_pet_enj...,0.000398,0.00006,6.584319,0.0,product_feedback =~ ratings_reviews_good_exper...,0.954372,0.936121,0.226292,TEMPTATIONS,CAT FOOD,19
